# Google's Quick Draw Dataset and GANs
## Lawton Manning, John Farrell, and Liyao Zhang

*Before running this Notebook:* <br> Make sure to download the QuickDraw files corresponding to the '.npy' filetype.

More information on Google's Quick Draw dataset found in their repository here: https://github.com/googlecreativelab/quickdraw-dataset 

## Preliminary Data Prep:

In [16]:
import numpy as np
import pandas as pd

from os import listdir
from os.path import basename, isfile, join

from matplotlib.pyplot import imshow

In [2]:
metadata = pd.read_csv(join('data', 'metadata.csv'))

In [54]:
%reload_ext autoreload
%autoreload 2
from DataGenerator import DataGenerator

datagen = DataGenerator(metadata, 32)   # locally defined Datagen class specific to Google's QuickDraw
datagen.n_classes

345

## Creating the GAN Model:
In order to model our GAN, we must define the two sub-models that comprise GANs in general. Collectively, they will learn to create sketches from Google's QuickDraw 28x28 greyscale images.
    
1. Discriminator - given an input image, determine if the image is real or fake
2. Generator - learn how to generate an image which fools the discriminator (so that it cannot distinguish real/fake reliably)
    

In [86]:
from keras.models import Sequential, Model
from keras import Input
from keras.layers import Dense
from keras.layers import Reshape, Flatten
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, BatchNormalization, Dropout
from keras.layers import Embedding

from keras.optimizers import Adam

def discriminator():
    dropout=0.4
    depth=64
    alpha=0.2
    
    label = Input(shape=(1,))
    x = Embedding(345,50)(label)
    x = Dense(784)(x)
    l = Reshape((28,28,1))(x)
    
    image = Input(shape=(28,28,1))
    
    concat = Concatenate()([image,l])
    x = Conv2D(1*depth,5,strides=2,padding='same')(concat)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(2*depth,5,strides=2,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(4*depth,5,strides=2,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(8*depth,5,strides=1,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Flatten()(x)
    out = Dense(1,activation='sigmoid')(x)
    
    model = Model(inputs=[image,label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def generator():
    dropout=0.4
    depth=32
    alpha=0.2
    
    label = Input(shape=(1,))
    x = Embedding(345,50)(label)
    x = Dense(49)(x)
    l = Reshape((7,7,1))(x)
    
    
    z = Input(shape=(100,))
    x = Dense((8*depth-1)*49)(z)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Dropout(dropout)(x)
    image = Reshape((7,7,8*depth-1))(x)
    
    concat = Concatenate()([image,l])
    x = UpSampling2D()(concat)
    x = Conv2DTranspose(4*depth,5,padding='same')(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = UpSampling2D()(x)
    x = Conv2DTranspose(2*depth,5,padding='same')(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2DTranspose(1*depth,5,padding='same')(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(alpha=alpha)(x)
    out = Conv2DTranspose(1,5,padding='same',activation='sigmoid')(x)
    
    model = Model(inputs=[z,label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

Model: "functional_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_36 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 1, 50)        17250       input_36[0][0]                   
__________________________________________________________________________________________________
dense_30 (Dense)                (None, 1, 784)       39984       embedding_17[0][0]               
__________________________________________________________________________________________________
input_37 (InputLayer)           [(None, 28, 28, 1)]  0                                            
______________________________________________________________________________________

In [88]:
model = generator()
model.summary()

Model: "functional_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
dense_33 (Dense)                (None, 12495)        1261995     input_39[0][0]                   
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 12495)        49980       dense_33[0][0]                   
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, 1)]          0                                            
______________________________________________________________________________________